### Importando Pacotes e Bibliotecas

In [187]:
import pandas as pd
import zipfile
import os
import numpy as np

# 1. Funções de Processamento dos Dataframes SIM e SINASC Separados

In [158]:
def obter_caminhos_dataframes(pasta):
    caminhos = []
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo = os.path.join(pasta, nome_arquivo)
        if os.path.isfile(caminho_arquivo) and nome_arquivo.endswith(".csv"):
            caminhos.append(caminho_arquivo)
    return caminhos

In [46]:
def remove_colunas_linhas_nan(dfs_paths):
    dfs_sem_colunas_nan = []

    for path in dfs_paths:
        # Verifica se o arquivo existe
        if not os.path.isfile(path):
            print(f"O arquivo {path} não existe.")
            continue

        # Carrega o DataFrame a partir do arquivo
        df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel

        # Remove colunas que contêm apenas NaN
        df = df.dropna(axis=1, how='all')

         # Removelinhas que contêm apenas NaN
        df = df.dropna(axis=0, how='all')

        # Armazena o DataFrame processado na lista
        dfs_sem_colunas_nan.append(df)

    # Retorna a lista de DataFrames processados
    return dfs_sem_colunas_nan

In [47]:
def remove_linhas_duplicatas(lista_dataframes):
    dfs_sem_duplicatas = []

    for df in lista_dataframes:
        # Remove as linhas duplicatas do DataFrame
        df = df.drop_duplicates()

        # Adiciona o DataFrame processado à lista
        dfs_sem_duplicatas.append(df)

    return dfs_sem_duplicatas

In [48]:
def remove_linhas_por_tipo_obito(lista_dataframes):
    dataframes_processados = []

    for df in lista_dataframes:
        # Remove as linhas onde a coluna 'TIPObito' é igual a 1
        df_processado = df[df['TIPOBITO'] != 1]

        # Adiciona o DataFrame processado à lista
        dataframes_processados.append(df_processado)

    return dataframes_processados
#dataframes_sem_tipo_obito_1 = remove_linhas_por_tipo_obito(dfs_sem_duplicatas)

In [49]:
def remove_linhas_menor_que_2018(lista_dfs, coluna_ano_nasc):
    # Lista para armazenar os DataFrames atualizados
    dfs_atualizados = []

    for df in lista_dfs:
        # Remover as linhas onde a coluna 'ano_nasc' é menor que 2018
        df = df[df[coluna_ano_nasc] >= 2018]

        # Adicionar o DataFrame atualizado à lista
        dfs_atualizados.append(df)

    return dfs_atualizados

In [51]:
def converte_coluna_float_para_int_e_depois_str(lista_dataframes, nome_coluna):
    for df in lista_dataframes:
        # Verifica se a coluna existe no DataFrame
        if nome_coluna in df.columns:
            # Converte a coluna para Int64 e depois para str
            df[nome_coluna] = df[nome_coluna].astype('Int64').astype(str)
        else:
            print(f"A coluna '{nome_coluna}' não foi encontrada no DataFrame.")

    return lista_dataframes

#lista_dfs_coluna_tipo_convertido = converte_coluna_float_para_str(dataframes_sem_tipo_obito_1, 'NATURAL')

In [52]:
def adiciona_coluna_nasc_NOME_UF_exclui_NATURAL_e_exclui_outros_paises(lista_dataframes):
    def mapear_estado(codigo):
        if codigo.startswith('8') and len(codigo) >= 3:
            codigo_estado = codigo[1:3]
            return {
                '11': 'RO', '12': 'AC', '13': 'AM', '14': 'RR', '15': 'PA', '16': 'AP', '17': 'TO',
                '21': 'MA', '22': 'PI', '23': 'CE', '24': 'RN', '25': 'PB', '26': 'PE', '27': 'AL',
                '28': 'SE', '29': 'BA', '31': 'MG', '32': 'ES', '33': 'RJ', '35': 'SP', '41': 'PR',
                '42': 'SC', '43': 'RS', '50': 'MS', '51': 'MT', '52': 'GO', '53': 'DF'
            }.get(codigo_estado, codigo)
        else:
            return codigo

    dataframes_processados = []

    for df in lista_dataframes:
        # Exclui as linhas cujo valor da coluna 'NATURAL' não começa com '8'
        df = df[df['NATURAL'].astype(str).str.startswith('8')]

        # Cria a nova coluna 'nasc_NOME_UF' com base na coluna 'NATURAL'
        df['nasc_NOME_UF'] = df['NATURAL'].apply(mapear_estado)

        # Exclui a coluna 'NATURAL'
        if 'NATURAL' in df.columns:
            df.drop('NATURAL', axis=1, inplace=True)

        # Adiciona DataFrame processado à lista
        dataframes_processados.append(df)

    return dataframes_processados
#lista_coluna_adicionada = adiciona_coluna_nasc_NOME_UF_exclui_NATURAL_e_exclui_outros_paises(lista_dfs_coluna_tipo_convertido)

In [53]:
def converte_coluna_para_inteiro(lista_dfs, coluna):
    # Lista para armazenar os DataFrames atualizados
    dfs_atualizados = []

    for df in lista_dfs:
        # Converter a coluna especificada para inteiro
        try:
            df[coluna] = pd.to_numeric(df[coluna]).astype('Int64')
            print(f"Coluna '{coluna}' convertida para inteiro em DataFrame.")
        except ValueError as e:
            print(f"Erro ao converter coluna '{coluna}' para inteiro em DataFrame: {e}")
            continue

        # Adicionar o DataFrame atualizado à lista
        dfs_atualizados.append(df)

    return dfs_atualizados

In [11]:
def salva_dataframes_com_nomes_SIM(lista_dataframes, pasta_destino):
    for df in lista_dataframes:
        # Criar o nome do arquivo combinando 'ano_obito' e 'res_SIGLA_UF'
        nome_arquivo = f"SIM_{df['ano_obito'].iloc[0]}_{df['res_SIGLA_UF'].iloc[0]}.csv"

        # Caminho completo para salvar o arquivo
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)

        # Salvar o DataFrame como arquivo CSV
        df.to_csv(caminho_arquivo, index=False)

In [54]:
def salva_dataframes_com_nomes_SINASC(lista_dataframes, pasta_destino):
    for df in lista_dataframes:
        # Criar o nome do arquivo combinando 'ano_obito' e 'res_SIGLA_UF'
        nome_arquivo = f"SINASC_{df['ano_nasc'].iloc[0]}_{df['res_SIGLA_UF'].iloc[0]}.csv"

        # Caminho completo para salvar o arquivo
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)

        # Salvar o DataFrame como arquivo CSV
        df.to_csv(caminho_arquivo, index=False)

# 2. Processamento

## 2.1 SIM

In [147]:
caminhos_dfs = obter_caminhos_dataframes('SIM')

In [16]:
lista1_SIM = remove_colunas_linhas_nan(caminhos_dfs)

C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (49,58,60,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppDa

In [17]:
lista2_SIM = remove_linhas_duplicatas(lista1_SIM)

In [19]:
lista3_SIM = remove_linhas_por_tipo_obito(lista2_SIM)

In [20]:
lista4_SIM = remove_linhas_menor_que_2018(lista3_SIM,'ano_nasc')

In [23]:
lista5_SIM = converte_coluna_float_para_int_e_depois_str(lista4_SIM, 'NATURAL')

C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\112744254.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nome_coluna] = df[nome_coluna].astype('Int64').astype(str)
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\112744254.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nome_coluna] = df[nome_coluna].astype('Int64').astype(str)
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\112744254.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [29]:
lista6_SIM = adiciona_coluna_nasc_NOME_UF_exclui_NATURAL_e_exclui_outros_paises(lista5_SIM)

C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\275308674.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nasc_NOME_UF'] = df['NATURAL'].apply(mapear_estado)
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\275308674.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('NATURAL', axis=1, inplace=True)
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\275308674.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [34]:
lista7_SIM = converte_coluna_para_inteiro(lista6_SIM, 'DTNASC')

Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida para inteiro em DataFrame.
Coluna 'DTNASC' convertida p

C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\262196295.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = pd.to_numeric(df[coluna]).astype('Int64')


In [36]:
lista8_SIM = converte_coluna_para_inteiro(lista7_SIM, 'ano_nasc')

Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em DataFrame.
Coluna 'ano_nasc' convertida para inteiro em Dat

C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\262196295.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = pd.to_numeric(df[coluna]).astype('Int64')


In [37]:
salva_dataframes_com_nomes_SIM(lista8_SIM, 'SIM_Processados')

In [157]:
#Limpa variáveis do notebook
%reset -f


## 2.2 SINASC

In [55]:
caminhos_dfs_SINASC = obter_caminhos_dataframes('SINASC')

In [56]:
lista1_SINASC = remove_colunas_linhas_nan(caminhos_dfs_SINASC)

C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\2401873829.py:11: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)  # ou pd.read_excel(path) se for um arquivo Excel
C:\Users\Elke\AppData\Local

In [57]:
lista2_SINASC = remove_linhas_duplicatas(lista1_SINASC)

In [58]:
salva_dataframes_com_nomes_SINASC(lista2_SINASC, 'SINASC_Processados')

In [85]:
#Limpa variáveis do notebook
%reset -f


# 3. Selecionando Chave

In [106]:
# Carrega a base de dados
df_SIM = pd.read_csv('SIM_Processados\\SIM_2018_AC.csv')  # ou pd.read_csv('dados.csv') para Pandas
df_SINASC = pd.read_csv('SINASC_Processados\\SINASC_2018_AC.csv')  # ou pd.read_csv('dados.csv') para Pandas


C:\Users\Elke\AppData\Local\Temp\ipykernel_12204\35669873.py:3: DtypeWarning: Columns (112) have mixed types. Specify dtype option on import or set low_memory=False.
  df_SINASC = pd.read_csv('SINASC_Processados\\SINASC_2018_AC.csv')  # ou pd.read_csv('dados.csv') para Pandas


In [65]:
# Obtenha os nomes das colunas de ambos os DataFrames
colunas_df1 = set(df_SIM.columns)
colunas_df2 = set(df_SINASC.columns)

# Encontre a interseção (colunas comuns) entre os dois conjuntos
colunas_comuns = colunas_df1.intersection(colunas_df2)

# Imprima ou faça o que quiser com as colunas comuns
print("Colunas em comum:", colunas_comuns)


Colunas em comum: {'res_NOME_UF', 'res_FRONTEIRA', 'res_LATITUDE', 'QTDFILVIVO', 'def_parto', 'QTDFILMORT', 'def_raca_cor', 'res_AREA', 'SEMAGESTAC', 'nasc_NOME_UF', 'GESTACAO', 'res_MSAUDCOD', 'SERIESCMAE', 'GRAVIDEZ', 'PARTO', 'def_est_civil', 'ORIGEM', 'CODMUNNATU', 'ESCMAE2010', 'res_REGIAO', 'CODMUNRES', 'DTCADASTRO', 'DTRECEBIM', 'res_CAPITAL', 'VERSAOSIST', 'CONTADOR', 'res_MUNNOMEX', 'DTNASC', 'CODESTAB', 'res_coordenadas', 'res_CODIGO_UF', 'IDADEMAE', 'ESCMAE', 'res_codigo_adotado', 'SEXO', 'res_RSAUDCOD', 'ano_nasc', 'RACACOR', 'PESO', 'ESCMAEAGR1', 'dia_semana_nasc', 'DIFDATA', 'def_sexo', 'res_LONGITUDE', 'res_ALTITUDE', 'data_nasc', 'res_SIGLA_UF', 'res_MUNNOME', 'def_escol_mae', 'res_AMAZONIA', 'res_CSAUDCOD', 'def_gestacao', 'def_gravidez', 'NUMEROLOTE'}


In [66]:
# Verifica se as colunas comuns têm o mesmo tipo de dados
for coluna in colunas_comuns:
    tipo_df1 = df_SINASC[coluna].dtype
    tipo_df2 = df_SIM[coluna].dtype

    if tipo_df1 != tipo_df2:
        print(f"A coluna {coluna} tem tipos diferentes: {tipo_df1} em df1 e {tipo_df2} em df2.")

In [ ]:
#------------   Análise das Colunas em Comum   ----------------

#Colunas Com Informações sobre Residência:
'res_NOME_UF', 
'res_FRONTEIRA', 
'res_LATITUDE',
'res_MSAUDCOD', 
'res_REGIAO',
'res_MUNNOMEX',
'res_coordenadas', 
'res_CODIGO_UF',
'res_codigo_adotado',
'res_SIGLA_UF',
'res_MUNNOME', 
'res_AMAZONIA', 
'res_CSAUDCOD',
'res_LONGITUDE',
'res_ALTITUDE', 
'res_RSAUDCOD',
'res_AREA',
'res_CAPITAL'

#Colunas das Informações sobre a Mãe do Falecido ou Nascido:
'QTDFILVIVO',   
'QTDFILMORT', 
'SERIESCMAE',
'ESCMAE2010',
'ESCMAE',
'IDADEMAE',
'ESCMAEAGR1',
'def_escol_mae',
'def_est_civil'  # (Não há descrição se é da mãe ou do indivíduo, como trata-se de bebês devido aos anos de análise deve ser o da mãe)

#Colunas das Informações sobre o Indivíduo Nascido ou Falecido
'SEMAGESTAC', 
'GESTACAO',  
'GRAVIDEZ', 
'PARTO',  
'DTNASC',
'SEXO',
'CODMUNNATU', 
'DTCADASTRO', 
'RACACOR',
'PESO',  
'dia_semana_nasc',  
'def_sexo', 
'data_nasc',   
'def_gestacao',
'def_gravidez',
'def_parto',
'def_raca_cor',
'ano_nasc', 
'nasc_NOME_UF'  # (criada no processamento) 

#Colunas com Significados Diferentes ou Indefinidos
'ORIGEM'  # (sem descrição)
'CONTADOR'  # (sem descrição)
'VERSAOSIST'  # (sem significado)
'CODESTAB'  # (sem significado)
'NUMEROLOTE'  # (sem significado)
'DTRECEBIM'  # (sem significado)
'CODMUNRES'  # (significados diferentes - na SIM é o município de residência do falecido e na SINASC da mãe)
'DIFDATA'  # (significados diferentes - na SIM é diferença entre a data de óbito e data do recebimento original da DO ([DTOBITO] – [DTRECORIG]) e na SINASC Diferença entre a data de óbito e data do recebimento original da DO ([DTNASC] – [DTRECORIG]), ou seja, as equações são diferentes, então não sabe-se se realmente são iguais)
'DTCADASTRO'  # (significados diferentes - descrições diferentes)
'CODMUNNATU'  # (significados diferentes - na SIM é o município de naturalidade do falecido e na sinasc da mãe do nascido)

Na chave utilizei apenas informações a respeito do indivíduo pois informações sobre a mãe ou residência podem ter sido alteradas entre o nascimento e o óbito, logo, podem estar diferentes nos registros e serem inconsistentes para a junção das bases.

As colunas com mesmo nome, mas com significados diferentes também não são úteis para compor a chave, bem como as colunas que não tem significados aparentemente, podendo até serem removidas antes da junção para reduzir o tempo de processamento, visto que as bases são grandes.

In [160]:
chave = ['def_gestacao','def_gravidez', 'def_parto','data_nasc','def_sexo', 'def_raca_cor','PESO','dia_semana_nasc'] 

# 4.Concatenando Dataframes

In [161]:

def concatenar_dataframes(lista_caminhos_dfs):
    # Lista para armazenar os DataFrames lidos
    dfs = []

    # Iterar sobre os caminhos dos arquivos
    for caminho_arquivo in lista_caminhos_dfs:
        # Ler o DataFrame a partir do arquivo CSV
        df = pd.read_csv(caminho_arquivo)

        # Adicionar o DataFrame à lista
        dfs.append(df)

    # Concatenar os DataFrames na lista em um único DataFrame
    df_concatenado = pd.concat(dfs, ignore_index=True)

    return df_concatenado

## 4.1 SIM


In [151]:
caminho_SIM_Processados = obter_caminhos_dataframes('SIM_Processados')

In [152]:
SIM_concatenada = concatenar_dataframes(caminho_SIM_Processados)

C:\Users\Elke\AppData\Local\Temp\ipykernel_15876\591623562.py:8: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo)


## 4.2 SINASC

In [162]:
caminho_SINASC_Processados = obter_caminhos_dataframes('SINASC_Processados')

In [163]:
SINASC_concatenada = concatenar_dataframes(caminho_SINASC_Processados)

C:\Users\Elke\AppData\Local\Temp\ipykernel_15876\591623562.py:8: DtypeWarning: Columns (112) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo)
C:\Users\Elke\AppData\Local\Temp\ipykernel_15876\591623562.py:8: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo)
C:\Users\Elke\AppData\Local\Temp\ipykernel_15876\591623562.py:8: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo)
C:\Users\Elke\AppData\Local\Temp\ipykernel_15876\591623562.py:8: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo)
C:\Users\Elke\AppData\Local\Temp\ipykernel_15876\591623562.py:8: DtypeWarning: Columns (112) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(camin

## 5. Retirando Colunas Sem Relevância

In [164]:
colunas_sem_significado = ['ORIGEM','CONTADOR','VERSAOSIST','CODESTAB','NUMEROLOTE','DTRECEBIM']
#colunas_redundantes = ['def_sexo', 'data_nasc', 'def_gestacao','def_gravidez','def_parto','def_raca_cor']
#colunas_redundantes =  ['SEMAGESTAC', 'GESTACAO','GRAVIDEZ', 'PARTO','DTNASC','SEXO', 'RACACOR']

In [165]:
colunas_para_dropar = colunas_sem_significado

In [155]:
SIM_concatenada.drop(colunas_para_dropar, axis=1,  inplace=True)

In [156]:
#SIM_concatenada.to_csv('SIM_concatenada.csv')

In [166]:
SINASC_concatenada.drop(colunas_para_dropar, axis=1,  inplace=True)

In [167]:
#SINASC_concatenada.to_csv('SINASC_concatenada.csv')

# 6. Merge

In [169]:
SIM_SINASC = pd.merge(SIM_concatenada, SINASC_concatenada, on = chave, how = 'inner')

In [170]:
SIM_SINASC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107113 entries, 0 to 107112
Columns: 252 entries, Unnamed: 0 to res_coordenadas_y
dtypes: float64(91), int64(71), object(90)
memory usage: 205.9+ MB


In [176]:
SIM_SINASC.head()

,Unnamed: 0,TIPOBITO,DTOBITO,HORAOBITO,CODMUNNATU_x,DTNASC_x,IDADE,idade_obito_anos,idade_obito_meses,idade_obito_dias,...,res_REGIAO_y,ano_nasc_y,parto_prematuro,def_parto_prematuro,codanomal_capitulo,codanomal_grupo,codanomal_categoria,codanomal_subcategoria,nasc_coordenadas,res_coordenadas_y
0,4,2,24062018,1740.0,120032.0,23062018,120,0.0,NaN,NaN,...,Norte,2018,0.0,Termo,NaN,NaN,NaN,NaN,"-9.434,-71.884","-9.434,-71.884"
1,4,2,24062018,1740.0,120032.0,23062018,120,0.0,NaN,NaN,...,Centro-Oeste,2018,0.0,Termo,NaN,NaN,NaN,NaN,"-14.516,-52.895","-14.516,-52.895"
2,9,2,6032018,1305.0,120030.0,6032018,5,0.0,NaN,NaN,...,Norte,2018,3.0,Prematuro,XVII.Malf cong deformid e anomalias cromossômicas,Malformações congênitas dos órgãos genitais,Q56 Sexo indeterminado e pseudo-hermafroditismo,NaN,"-8.164,-70.354","-8.164,-70.354"
3,10,2,10032018,1645.0,120030.0,9012018,302,0.0,2.0,NaN,...,Norte,2018,0.0,Termo,NaN,NaN,NaN,NaN,"-8.164,-70.354","-8.164,-70.354"
4,18,2,13012018,400.0,120020.0,9012018,204,0.0,NaN,4.0,...,Norte,2018,3.0,Prematuro,NaN,NaN,NaN,NaN,"-7.631,-72.67","-7.631,-72.67"


In [184]:
dropar = ['Unnamed: 0','TIPOBITO']
SIM_SINASC.drop(dropar, axis=1,  inplace=True)

In [185]:
#SIM_SINASC.to_csv('SIM_SINASC_2018_ate_2021.csv')

In [186]:
SIM_SINASC.head()

,DTOBITO,HORAOBITO,CODMUNNATU_x,DTNASC_x,IDADE,idade_obito_anos,idade_obito_meses,idade_obito_dias,idade_obito_horas,idade_obito_mins,...,res_REGIAO_y,ano_nasc_y,parto_prematuro,def_parto_prematuro,codanomal_capitulo,codanomal_grupo,codanomal_categoria,codanomal_subcategoria,nasc_coordenadas,res_coordenadas_y
0,24062018,1740.0,120032.0,23062018,120,0.0,NaN,NaN,20.0,NaN,...,Norte,2018,0.0,Termo,NaN,NaN,NaN,NaN,"-9.434,-71.884","-9.434,-71.884"
1,24062018,1740.0,120032.0,23062018,120,0.0,NaN,NaN,20.0,NaN,...,Centro-Oeste,2018,0.0,Termo,NaN,NaN,NaN,NaN,"-14.516,-52.895","-14.516,-52.895"
2,6032018,1305.0,120030.0,6032018,5,0.0,NaN,NaN,NaN,5.0,...,Norte,2018,3.0,Prematuro,XVII.Malf cong deformid e anomalias cromossômicas,Malformações congênitas dos órgãos genitais,Q56 Sexo indeterminado e pseudo-hermafroditismo,NaN,"-8.164,-70.354","-8.164,-70.354"
3,10032018,1645.0,120030.0,9012018,302,0.0,2.0,NaN,NaN,NaN,...,Norte,2018,0.0,Termo,NaN,NaN,NaN,NaN,"-8.164,-70.354","-8.164,-70.354"
4,13012018,400.0,120020.0,9012018,204,0.0,NaN,4.0,NaN,NaN,...,Norte,2018,3.0,Prematuro,NaN,NaN,NaN,NaN,"-7.631,-72.67","-7.631,-72.67"
